In [1]:
import lightkurve as lk
import pandas as pd
import numpy as np
import pickle
import feets
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from astropy.stats import lombscargle

In [2]:
init_file = pd.read_csv('Rafael_Sample.csv', sep=';')
class_df = pd.DataFrame()
table = pd.DataFrame()

for i in range(len(init_file)):
    search = lk.search_lightcurve(target=f'TIC {init_file.iloc[i, 0]}', author='SPOC', exptime=120)
    for j in range(len(search)):
        sector = search.mission[j][12:]
        lc = search[j].download(download_dir='E:/lightkurve_fits/').normalize().remove_outliers(sigma=3.5)
        
        lc_feets = [lc.time.value, lc.flux.value, lc.flux_err]
        fs = feets.FeatureSpace(data=['magnitude', 'time', 'error'], only=['Amplitude', 'LinearTrend', 'MaxSlope', 'Mean', 'Meanvariance', 'MedianAbsDev', 'Std', 'PeriodLS', 'Period_fit', 'Psi_CS', 'Psi_eta'])
        warnings.filterwarnings("ignore")
        features, values = fs.extract(*lc_feets)
        
        feets_table = pd.DataFrame(data=[values], columns=[features], index=[f'{init_file.iloc[i, 0]}_{sector}'])
        table = pd.concat([table, feets_table], axis=0)
        temp_table = pd.DataFrame(data=[init_file.iloc[i, 1]], columns=['Class'])
        class_df = pd.concat([class_df, temp_table], axis=0, ignore_index=True)
        
table = table.rename_axis().reset_index()
data_table = pd.concat([table, class_df], axis=1, ignore_index=True).rename({0:'TIC_Sect', 1:'Amplitude', 2:'LinearTrend', 3:'MaxSlope', 4:'Mean', 5:'Meanvariance', 6:'MedianAbsDev', 7:'Std', 8:'PeriodLS', 9:'Period_fit', 10:'Psi_CS', 11:'Psi_eta', 12:'Class'}, axis=1)
data_table

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [26]:
np.shape(data_table)

(5, 21)